In [ ]:
import numpy as np
from jesse import helpers, research

_, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

trading_1m.shape

In [ ]:
np.save("data/btc_1m.npy", trading_1m)

In [1]:
import numpy as np
from jesse import helpers, research

trading_1m = np.load("data/btc_1m.npy")
trading_1m.shape

(2674080, 6)

In [2]:
from custom_indicators.toolbox.dollar_bar import estimate_dollar_bar_threshold

for i in [10, 25, 60 * 2]:
    threshold = estimate_dollar_bar_threshold(trading_1m, i)
    print(f"{i}m: {threshold}")

10m: 71968406.59988706
25m: 203617929.03870487
120m: 1039153564.783735


In [4]:
from custom_indicators.config import (
    DOLLAR_BAR_THRESHOLD_2H,
    DOLLAR_BAR_THRESHOLD_10M,
    DOLLAR_BAR_THRESHOLD_25M,
)
from custom_indicators.toolbox.dollar_bar import build_dollar_bar

dollar_bar_10m = build_dollar_bar(trading_1m, DOLLAR_BAR_THRESHOLD_10M)
dollar_bar_25m = build_dollar_bar(
    trading_1m, DOLLAR_BAR_THRESHOLD_25M
)  # for side label
dollar_bar_2h = build_dollar_bar(trading_1m, DOLLAR_BAR_THRESHOLD_2H)
print(dollar_bar_10m.shape, dollar_bar_25m.shape, dollar_bar_2h.shape)

np.save("data/dollar_bar_10m.npy", dollar_bar_10m)
np.save("data/dollar_bar_2h.npy", dollar_bar_2h)

(268413, 6) (106174, 6) (22243, 6)


In [6]:
from custom_indicators.config import DOLLAR_BAR_THRESHOLD_25M
from custom_indicators.toolbox.dollar_bar import build_dollar_bar

candles_dollar_bar = build_dollar_bar(trading_1m, DOLLAR_BAR_THRESHOLD_25M)
candles_dollar_bar.shape

(106174, 6)

In [7]:
from custom_indicators.toolbox.labeler.zigzag_labeler import zigzag

res = zigzag(candles_dollar_bar[:, 2], 0.01)
np.unique(res, return_counts=True)

(array([-1,  1]), array([51154, 55020]))

In [8]:
label = np.column_stack((candles_dollar_bar, res))
label

array([[ 1.57787454e+12,  7.20098000e+03,  7.19050000e+03, ...,
         7.18501000e+03,  2.81957600e+04, -1.00000000e+00],
       [ 1.57791852e+12,  7.19001000e+03,  7.12374000e+03, ...,
         7.11206000e+03,  2.85109630e+04, -1.00000000e+00],
       [ 1.57794900e+12,  7.12380000e+03,  7.11082000e+03, ...,
         7.10100000e+03,  2.86050300e+04, -1.00000000e+00],
       ...,
       [ 1.73827176e+12,  1.04961400e+05,  1.05061000e+05, ...,
         1.04864200e+05,  2.00223200e+03, -1.00000000e+00],
       [ 1.73827518e+12,  1.05060900e+05,  1.05209100e+05, ...,
         1.04997700e+05,  1.94644700e+03, -1.00000000e+00],
       [ 1.73827968e+12,  1.05209100e+05,  1.04676900e+05, ...,
         1.04628300e+05,  1.96454500e+03, -1.00000000e+00]])

In [9]:
np.save("data/label_side_25m.npy", label)

In [10]:
from jesse import utils

df_dollar_bar = utils.numpy_candles_to_dataframe(candles_dollar_bar, name_date="time")
df_dollar_bar["zigzag"] = res
df_dollar_bar.head(1)

,time,open,close,high,low,volume,zigzag
2020-01-01 10:29:00,2020-01-01 10:29:00,7200.98,7190.5,7260.43,7185.01,28195.76,-1


In [11]:
from lightweight_charts import Chart

chart = Chart(inner_width=1, inner_height=0.8)
chart2 = chart.create_subchart(position="bottom", width=1, height=0.2, sync=True)

chart.set(df_dollar_bar[["time", "open", "high", "low", "close", "volume"]])

line = chart2.create_line(name="zigzag", style="dotted")
line.set(df_dollar_bar[["time", "zigzag"]])

chart.show(block=True)

2025-03-08 09:42:04.732 python[91264:9498673] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-08 09:42:04.732 python[91264:9498673] +[IMKInputSession subclass]: chose IMKInputSession_Modern


RuntimeError: asyncio.run() cannot be called from a running event loop